In [1]:
!pip install beautifulsoup4


In [10]:
import json
import re

marker = '|'


def ingest_tanzil(filename, title): 
    surahs = []
    with open(filename, "r") as file:
        current_surah_idx = 0
        ayas = []
        for line in file:
            line_list = line.strip().split(marker)
            if(len(line_list) == 3):
                #print(line_list[0])
                if (line_list[0] != current_surah_idx):
                    if(current_surah_idx != 0):
                        #Finish Previous surah
                        surah = {
                            'title' : current_surah_idx,
                            'section' : ayas
                        }
                        surahs.append(surah)
                        ayas = []
                current_surah_idx = line_list[0]
                aya = {
                    'title' : f'{line_list[0]}:{line_list[1]}',
                    'section' : {
                        'title': title,
                        'text': re.sub('<[^<]+?>', '',line_list[2])
                    }
                }
                ayas.append(aya)
        # Finish naas
        #Finish Previous surah
        surah = {
            'title' : current_surah_idx,
            'section' : ayas
        }
        surahs.append(surah)
    return surahs

arabic = ingest_tanzil('../data/quran-uthmani-min.txt', "Arabic") 
sahih = ingest_tanzil('../data/en.sahih.txt', 'Saheeh International') 
translit = ingest_tanzil('../data/en.transliteration.txt', 'Transliteration') 

# Now combine every Ayah, using sahih as a template
new_surahs = []
for i in range(len(arabic)):
    print(f"Processing {arabic[i]['title']}")
    new_ayahs = []
    for j in range(len(arabic[i]['section'])):
        print(f"Processing {arabic[i]['section'][j]} {sahih[i]['section'][j]}")
        new_ayahs.append({ 'title': arabic[i]['section'][j]['title'], 
                           'section': [arabic[i]['section'][j]['section'], 
                                      sahih[i]['section'][j]['section'],
                                      translit[i]['section'][j]['section']]})
    new_surahs.append({'title': arabic[i]['title'],
                       'section': new_ayahs})
        


content = {
    'documentId': "Qur'an",
    'title' : "Qur'an",
    'section' : new_surahs


}
contents_json = json.dumps(content,separators=(',', ':'))
with open("vectara.json", "w") as output:
    output.write(contents_json)

Processing 1
Processing {'title': '1:1', 'section': {'title': 'Arabic', 'text': 'بِسمِ اللَّهِ الرَّحمٰنِ الرَّحيمِ'}} {'title': '1:1', 'section': {'title': 'Saheeh International', 'text': 'In the name of Allah, the Entirely Merciful, the Especially Merciful.'}}
Processing {'title': '1:2', 'section': {'title': 'Arabic', 'text': 'الحَمدُ لِلَّهِ رَبِّ العٰلَمينَ'}} {'title': '1:2', 'section': {'title': 'Saheeh International', 'text': '[All] praise is [due] to Allah, Lord of the worlds -'}}
Processing {'title': '1:3', 'section': {'title': 'Arabic', 'text': 'الرَّحمٰنِ الرَّحيمِ'}} {'title': '1:3', 'section': {'title': 'Saheeh International', 'text': 'The Entirely Merciful, the Especially Merciful,'}}
Processing {'title': '1:4', 'section': {'title': 'Arabic', 'text': 'مٰلِكِ يَومِ الدّينِ'}} {'title': '1:4', 'section': {'title': 'Saheeh International', 'text': 'Sovereign of the Day of Recompense.'}}
Processing {'title': '1:5', 'section': {'title': 'Arabic', 'text': 'إِيّاكَ نَعبُدُ وَإِيّ